<a href="https://colab.research.google.com/github/simransirsat/MLPracticals/blob/main/Gini_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**5A- Gini Index**

In [1]:
pip install anytree

     |████████████████████████████████| 51kB 2.1MB/s 


In [7]:
import pandas as pd

In [8]:
from anytree import RenderTree
from anytree import Node, RenderTree

In [9]:
df = pd.read_csv('/content/sample_data/data.csv')

In [5]:
def gini_index(df, attr):
    columns = {}
    for i in range(0, len(list(df.columns))):
        columns[list(df.columns)[i]] = i
    gini = {}
    for i in range(0, df.shape[0]):
        if df.iloc[i, columns[attr]] not in gini:
            gini[df.iloc[i, columns[attr]]] = {}
            gini[df.iloc[i, columns[attr]]][df.iloc[i,-1]] = 1
        elif df.iloc[i, -1] not in gini[df.iloc[i, columns[attr]]]:
            gini[df.iloc[i, columns[attr]]][df.iloc[i,-1]] = 1
        else:
            gini[df.iloc[i, columns[attr]]][df.iloc[i,-1]] += 1
    
    total_gini = 0
    for attr_label, target_label_cnts in gini.items():
        net_cnt = 0
        for label, cnt in target_label_cnts.items():
            net_cnt += cnt
        giniindex = 1
        for label, cnt in target_label_cnts.items():
            giniindex -= (cnt / net_cnt) ** 2
        print('\t', attr_label,' --> ', round(giniindex,4))
        total_gini += (net_cnt / df.shape[0]) * giniindex

    return round(total_gini,4)

def find_deciding_attr(df):
    min_gini = float('inf')
    min_gini_attr = None

    for i in range(0, df.shape[1] - 1):
        print(df.columns[i])
        gini = float(gini_index(df, df.columns[i]))
        print('GINI(',df.columns[i],') : ', round(gini,4))
        if gini < min_gini:
            min_gini = gini
            min_gini_attr = df.columns[i]
        elif gini == min_gini:
            if df.columns[i] < min_gini_attr:
                min_gini_attr = df.columns[i]

    print('Deciding attribute ', min_gini_attr,' with GINI INDEX: ', round(min_gini,4))

    return min_gini_attr

root = None

def make_decision_tree(df, parent = None, weight = None):
    global root    
    possible_target = {}
    print(df)
    for i in range(0, df.shape[0]):
        if df.iloc[i, -1] not in possible_target:
            possible_target[df.iloc[i, -1]] = True
    
    if (len(possible_target) == 1):
        node = Node(list(possible_target)[0], parent = parent, weight = str(weight))
        return

    if(df.shape[1] == 1 and len(possible_target) > 1):
        node = Node(list(possible_target)[0], parent = parent, weight = str(weight))
        return
    
    deciding_attr = find_deciding_attr(df)
    if deciding_attr is None:
        node = Node(df.iloc[:, -1].value_counts().idxmax(), parent = parent, weight = str(weight))
        return
        
    node = Node(deciding_attr, parent = parent, weight = str(weight))
    #print('df:', df)
    if (root is None):
        root = node
    
    columns = {}
    for i in range(0, len(list(df.columns))):
        columns[list(df.columns)[i]] = i
    
    possible_values = {}
    for i in range(0, df.shape[0]):
        if df.iloc[i, columns[deciding_attr]] not in possible_values:
            possible_values[df.iloc[i, columns[deciding_attr]]] = True

    #print('possible_values: ', possible_values)
    for attr_wt_label, _ in possible_values.items():
        print('Now checking for ',deciding_attr,':\t', attr_wt_label)
        cols = list(df.columns)
        cols.remove(deciding_attr)
        df_temp = pd.DataFrame(columns = cols)
        
        for i in range(0, df.shape[0]):
            if (df.iloc[i, columns[deciding_attr]] == attr_wt_label):
                row = df.iloc[i, :].tolist()
                row.pop(columns[deciding_attr])
                #df_temp.loc[len(df)] = row
                df_temp = df_temp.append(pd.DataFrame([row], columns = cols), ignore_index = True)
        make_decision_tree(df_temp, node, attr_wt_label)

    return

In [6]:
root = None
make_decision_tree(df)

print('The Decision tree based on Gini Index is')
for pre, _, node in RenderTree(root):
    print("%s%s (%s)" % (pre, node.name, node.weight))

   Restaurant       Meal       Day       Cost Vegan AllergicReaction?
0       Sam's  Breakfast  Saturday  Expensive   Yes               Yes
1       Kim's  Breakfast    Sunday      Cheap   Yes                No
2       Sam's      Lunch  Saturday      Cheap   Yes               Yes
3       Kim's  Breakfast    Sunday  Expensive    No                No
4       Sam's  Breakfast  Saturday      Cheap    No                No
5       Kim's      Lunch    Friday  Expensive    No               Yes
6       Kim's      Lunch  Saturday      Cheap   Yes                No
7       Kim's  Breakfast    Sunday  Expensive    No               Yes
8       Sam's  Breakfast  Saturday      Cheap    No                No
9       Kim's  Breakfast    Friday  Expensive    No               Yes
10      Sam's      Lunch  Saturday      Cheap    No                No
11      Kim's      Lunch    Friday      Cheap    No                No
Restaurant
	 Sam's  -->  0.48
	 Kim's  -->  0.4898
GINI( Restaurant ) :  0.4857
Meal
	 Bre